In [1]:
%run common.ipynb

In [2]:
np.random.seed(3)
random.seed(3)

In [3]:
####### Global notebook configs #######

# Toggle for enabling/disabling the 
# decorator
if measure_time.enabled:
    print("* measure_time is enabled ")

# specify the folder path and files name 
dataset_file_path = os.path.join('../datasets', 'cities.csv')
print(f"* the selected dataset is located at: {dataset_file_path}")

# select the nb of cities you want from the dataset
np_of_cities = 50
print(f"* {np_of_cities} will be used from the dataset")

# select the number of truck you wish to divide the workload
truck_nb = 5
print(f"* {truck_nb} will be used to deliver the goods")

# choose an average speed that suit your needs (here 51.3 km/h)
average_speed = 51.3
print(f"* the average speed is : {average_speed} km/h")

# tabu related conf 
num_iterations = 3000
tabu_list_size = 1000
print(f"* the tabu list size is : {tabu_list_size} and it will iterate {num_iterations} times")

# multistart related conf
num_starts = 10
print(f"* multistart factor : {num_starts}")

####### ####### ####### ####### #######

* measure_time is enabled 
* the selected dataset is located at: ../datasets/cities.csv
* 50 will be used from the dataset
* 5 will be used to deliver the goods
* the average speed is : 51.3 km/h
* the tabu list size is : 1000 and it will iterate 3000 times
* multistart factor : 10


# City Generator 

this part contains the folowing logic: we first retrieve data from a dataset and later construct a sample from it that contain the cities name, the ZIP Code, the population count and longitude|latitude 

In [4]:
# disable performance profiling for this section 
measure_time.enabled = False

citiesTuple = read_csv_to_tuple(dataset_file_path)
citiesSample = sample_N_from_tuple(citiesTuple, np_of_cities)

# display the map with the selected cities
plot_cities(citiesSample)

# location generator 
The purpose of this staged is to generate a series of city names along with their respective longitude and latitude coordinates. It achieves this by extracting the relevant information from a given list of city data

In [5]:
location = create_location_generator(citiesSample)

# disable performance profiling for this section 
measure_time.enabled = False

for city_name, coordinates in create_location_generator(citiesSample):
    print(f'{city_name}: {coordinates}')

Cernay-l'Église: (6.83407272, 47.24929776)
Allas-Bocage: (-0.50199900826446, 45.384684214876)
Issigeac: (0.60406452702703, 44.726417297297)
La Bourgonce: (6.8124869064748, 48.305416115108)
Saint-Mard: (3.5866173170732, 49.385179756098)
Cierp-Gaud: (0.63716026548673, 42.906808141593)
Saint-Géraud: (0.15511762886598, 44.624371958763)
Marcillé-Robert: (-1.35109640625, 47.95227203125)
Sembadel: (3.6860319230769, 45.275283605769)
Warluis: (2.1584695081967, 49.385440437158)
Plougastel-Daoulas: (-4.3814217726397, 48.356330366089)
Gravelotte: (6.0243075409836, 49.110500491803)
Courgent: (1.6586165151515, 48.896415)
Conte: (6.0070476271186, 46.750205762712)
Oëlleville: (6.024512972973, 48.334302342342)
Saint-Pierre-de-Bat: (-0.21850408163265, 44.670685714286)
Vatteville: (1.2861831067961, 49.28027961165)
Saint-Denis-d'Augerons: (0.47065918918919, 48.919018513514)
Curtil-sous-Buffières: (4.5281553900709, 46.401401985816)
Saint-Guinoux: (-1.8927695454545, 48.580484090909)
Uzech: (1.386475210084, 

# time matrix generator

this part calculate a time matrix for a set of cities based on their geographic coordinates. 

In [6]:
@measure_time
def calculate_time_matrix(generator) -> Dict[str, Dict[str, float]]:
    time_matrix = {}  # Create an empty dictionary to store the time matrix
    city_coords = []  # Create an empty list to store city names and coordinates
    
    # Iterate over each city name and coordinates from the generator and 
    # append the city name and coordinates as a tuple to city_coords
    for city_name, coordinates in generator:
        city_coords.append((city_name, coordinates)) 
    
    # Iterate over the city name and coordinates using enumerate
    for i, (city1, coords1) in enumerate(city_coords):
        
        # Create an empty dictionary for each city in the time matrix
        time_matrix[city1] = {}
        
        # Iterate over the city name and coordinates again
        for j, (city2, coords2) in enumerate(city_coords):  
            if i == j:
                # Set the time between a city and itself to 0.0
                time_matrix[city1][city2] = 0.0 
            else:
                # Calculate the geodesic distance between two coordinates
                distance = geodesic(coords1, coords2).kilometers
                # Store the time in the time matrix
                time_matrix[city1][city2] = distance / average_speed 
    
    return time_matrix

In [7]:
"""

The traffic matrix generator applies a random factor to the times 
in order to simulate the increased time it would take to travel from 
city A to city B due to traffic conditions. 

"""
@measure_time
def generate_traffic_matrix(time_matrix):
    traffic_matrix = {}

    for city_a, times in time_matrix.items():
        traffic_matrix[city_a] = {}

        for city_b, time in times.items():
            if city_a == city_b:
                traffic_matrix[city_a][city_b] = 1.0  # Assuming no traffic within the same city
            else:
                traffic_matrix[city_a][city_b] = np.random.uniform(1.0, 1.5) * time
    
    return traffic_matrix

In [8]:
# disable performance profiling for this section 
measure_time.enabled = False

time_matrix = calculate_time_matrix(
    create_location_generator(citiesSample)
)

traffic_matrix = generate_traffic_matrix(time_matrix)

# tabu algorithm

In [9]:
def tabu_search(time_matrix, num_iterations, tabu_list_size, start_town=None, progress_enable = True):
    # Initialize the tabu list as an empty set
    tabu_list = set()
    # Generate an initial random solution
    if start_town is None:
        best_route = list(time_matrix.keys())
        np.random.shuffle(best_route)
    else:
        best_route = list(time_matrix.keys())
        best_route.remove(start_town)
        best_route.insert(0, start_town)
    best_time = calculate_total_time(best_route, time_matrix)

    # Create a progress bar using tqdm
    if progress_enable: 
        progress_bar = tqdm(
            total=num_iterations, 
            desc="Tabu Search", 
            unit="iteration", 
        )

    # Start the iterations
    for _ in range(num_iterations):
        # Find the best neighboring solution
        neighbors = generate_neighbors(best_route, tabu_list)
        
        # [hack] Check if neighbors list is empty 
        if not neighbors:
            if progress_enable: 
                progress_bar.update(num_iterations - progress_bar.n)  # Force progress to 100%
            continue
        
        best_neighbor = min(neighbors, key=lambda x: calculate_total_time(x, time_matrix))

        # Update the best solution if the neighbor is an improvement
        neighbor_time = calculate_total_time(best_neighbor, time_matrix)
        if neighbor_time < best_time:
            best_route = best_neighbor
            best_time = neighbor_time

        # Add the best neighbor to the tabu list
        tabu_list.add(tuple(best_neighbor))
        # Remove the oldest solution from the tabu list if it exceeds the tabu list size
        if len(tabu_list) > tabu_list_size:
            tabu_list.remove(next(iter(tabu_list)))

        # Update the progress bar
        if progress_enable: 
            progress_bar.update(1)

    # Append the first town to the best route to complete the cycle
    best_route.append(best_route[0])
    best_time += time_matrix[best_route[-2]][best_route[0]]

    # Close the progress bar
    if progress_enable: 
        progress_bar.close()

    return best_route, best_time


def calculate_total_time(route, time_matrix):
    total_time = 0.0
    num_cities = len(route)
    for i in range(num_cities - 1):
        current_city = route[i]
        next_city = route[i + 1]
        total_time += time_matrix[current_city][next_city]
    return total_time


def generate_neighbors(route, tabu_list):
    route = np.array(route)
    tabu_list = set(map(tuple, tabu_list))
    num_cities = len(route)
    neighbors = []
    for i in range(1, num_cities):
        for j in range(i + 1, num_cities):
            neighbor = np.concatenate((route[:i], route[i:j][::-1], route[j:]))
            if tuple(neighbor) not in tabu_list:
                neighbors.append(neighbor.tolist())
    return neighbors

In [10]:
# without constrain 
best_route, best_time = tabu_search(time_matrix, num_iterations, tabu_list_size)
pprint_the_output(best_time, best_route)
plot_cities(citiesSample, best_route)

Tabu Search: 100%|████████████████████████████████████████████████████████████████████████████| 3000/3000 [00:47<00:00, 62.79iteration/s]

Best time: 108.0432696734068 h
Best Route: Oëlleville -> Oron -> Gravelotte -> Dannevoux -> Villiers-en-Lieu -> Marnay-sur-Marne -> Bar-sur-Aube -> Levis -> Jaignes -> Saint-Mard -> Boncourt -> Servais -> Crèvec½ur-sur-l'Escaut -> Linzeux -> Warluis -> Courgent -> Vatteville -> Heudreville-sur-Eure -> Saint-Denis-d'Augerons -> Bréauté -> Tessel -> Saint-Guinoux -> Plougastel-Daoulas -> Locqueltas -> Marcillé-Robert -> Billé -> Saint-Georges-du-Rosay -> Chapelle-Viviers -> Allas-Bocage -> Saint-Pierre-de-Bat -> Saint-Géraud -> Issigeac -> Cierp-Gaud -> Uzech -> Saint-Michel-de-Vax -> Saint-Saturnin-de-Lenne -> Maruéjols-lès-Gardon -> Sembadel -> Curtil-sous-Buffières -> Lacenas -> Saint-Martin-le-Vinoux -> Champagny-en-Vanoise -> Le Frasnois -> Conte -> Poinson-lès-Fayl -> Cernay-l'Église -> La Bourgonce -> Weinbourg -> Durrenbach -> Cleebourg -> Oëlleville


In [11]:
# with time constrain 
best_route, best_time = tabu_search(traffic_matrix, num_iterations, tabu_list_size)
pprint_the_output(best_time, best_route)
plot_cities(citiesSample, best_route)

Tabu Search: 100%|████████████████████████████████████████████████████████████████████████████| 3000/3000 [00:47<00:00, 62.88iteration/s]

Best time: 157.68596689442737 h
Best Route: Cleebourg -> Durrenbach -> Weinbourg -> La Bourgonce -> Oëlleville -> Oron -> Gravelotte -> Villiers-en-Lieu -> Bar-sur-Aube -> Levis -> Curtil-sous-Buffières -> Lacenas -> Sembadel -> Saint-Saturnin-de-Lenne -> Maruéjols-lès-Gardon -> Saint-Martin-le-Vinoux -> Champagny-en-Vanoise -> Cernay-l'Église -> Conte -> Le Frasnois -> Poinson-lès-Fayl -> Marnay-sur-Marne -> Dannevoux -> Boncourt -> Saint-Mard -> Jaignes -> Servais -> Crèvec½ur-sur-l'Escaut -> Linzeux -> Warluis -> Courgent -> Heudreville-sur-Eure -> Vatteville -> Bréauté -> Tessel -> Saint-Denis-d'Augerons -> Saint-Georges-du-Rosay -> Chapelle-Viviers -> Issigeac -> Uzech -> Saint-Michel-de-Vax -> Cierp-Gaud -> Saint-Géraud -> Saint-Pierre-de-Bat -> Allas-Bocage -> Saint-Guinoux -> Billé -> Plougastel-Daoulas -> Locqueltas -> Marcillé-Robert -> Cleebourg


# construct sub matrix for the truck repartition

In [12]:
def split_into_trucks(time_matrix, num_trucks):
    # Convert the time matrix to a numpy array
    towns = list(time_matrix.keys())
    time_array = np.array([[time_matrix[town1][town2] for town2 in towns] for town1 in towns])
    
    # Apply k-means clustering
    kmeans = KMeans(n_clusters=num_trucks, random_state=0, n_init=10).fit(time_array)
    labels = kmeans.labels_
    
    # Find the index of the first town in the time matrix
    first_town_index = towns.index(towns[0])
    
    # Create time matrices for each truck
    trucks = []
    for i in range(num_trucks):
        # Filter towns based on the label or if it's the first town
        truck_towns = [towns[j] for j in range(len(towns)) if labels[j] == i or j == first_town_index]
        truck_time_matrix = {town: {truck_town: time_matrix[town][truck_town] for truck_town in truck_towns} for town in truck_towns}
        trucks.append(truck_time_matrix)
    
    return trucks

def filter_cities_by_truck(citiesSample, truck):
    town_list = list(truck.keys())
    filtered_cities = [city for city in citiesSample if city[4] in town_list]
    return np.array(filtered_cities)


def optimize_trucks(trucks, map_obj, use_traffic_matrix=False):
    for i, truck in enumerate(trucks):
        if use_traffic_matrix:
            truck = generate_traffic_matrix(truck)
        best_route, best_time = tabu_search(truck, num_iterations, tabu_list_size, start_town=next(iter(truck)))
            
        print(f"The truck n: {i}")
        pprint_the_output(best_time, best_route)
        plot_cities(filter_cities_by_truck(citiesSample, truck), best_route, map_obj=map_obj, color=color[i])
    
    return map_obj

In [13]:
# withouth time constrain, only trucks
trucks = split_into_trucks(time_matrix, truck_nb)

map_obj = folium.Map(
    location=[
        float(citiesSample[0][2]), 
        float(citiesSample[0][3])
    ], 
    zoom_start=6)

optimize_trucks(trucks, map_obj)

Tabu Search: 100%|██████████████████████████████████████████████████████████████████████████| 3000/3000 [00:01<00:00, 2994.27iteration/s]


The truck n: 0
Best time: 41.87914952975381 h
Best Route: Cernay-l'Église -> Levis -> Jaignes -> Saint-Mard -> Boncourt -> Servais -> Warluis -> Courgent -> Saint-Georges-du-Rosay -> Saint-Denis-d'Augerons -> Bréauté -> Heudreville-sur-Eure -> Vatteville -> Linzeux -> Crèvec½ur-sur-l'Escaut -> Cernay-l'Église


Tabu Search: 100%|█████████████████████████████████████████████████████████████████████████| 3000/3000 [00:00<00:00, 18435.23iteration/s]


The truck n: 1
Best time: 24.201421266259278 h
Best Route: Cernay-l'Église -> Curtil-sous-Buffières -> Lacenas -> Sembadel -> Saint-Saturnin-de-Lenne -> Maruéjols-lès-Gardon -> Saint-Martin-le-Vinoux -> Champagny-en-Vanoise -> Cernay-l'Église


Tabu Search: 100%|█████████████████████████████████████████████████████████████████████████| 3000/3000 [00:00<00:00, 13315.59iteration/s]


The truck n: 2
Best time: 42.31188563132243 h
Best Route: Cernay-l'Église -> Uzech -> Saint-Michel-de-Vax -> Cierp-Gaud -> Issigeac -> Saint-Géraud -> Saint-Pierre-de-Bat -> Allas-Bocage -> Chapelle-Viviers -> Cernay-l'Église


Tabu Search: 100%|██████████████████████████████████████████████████████████████████████████| 3000/3000 [00:00<00:00, 3108.99iteration/s]


The truck n: 3
Best time: 23.070536027437164 h
Best Route: Cernay-l'Église -> Conte -> Le Frasnois -> Poinson-lès-Fayl -> Marnay-sur-Marne -> Bar-sur-Aube -> Villiers-en-Lieu -> Dannevoux -> Gravelotte -> Oron -> Oëlleville -> La Bourgonce -> Weinbourg -> Durrenbach -> Cleebourg -> Cernay-l'Église


Tabu Search: 100%|█████████████████████████████████████████████████████████████████████████| 3000/3000 [00:00<00:00, 27300.27iteration/s]

The truck n: 4
Best time: 51.23343658911645 h
Best Route: Cernay-l'Église -> Tessel -> Billé -> Marcillé-Robert -> Saint-Guinoux -> Plougastel-Daoulas -> Locqueltas -> Cernay-l'Église


In [14]:
# with time constrain and trucks
trucks = split_into_trucks(time_matrix, truck_nb)

map_obj = folium.Map(
    location=[
        float(citiesSample[0][2]), 
        float(citiesSample[0][3])
    ], 
    zoom_start=6)

optimize_trucks(trucks, map_obj)

Tabu Search: 100%|██████████████████████████████████████████████████████████████████████████| 3000/3000 [00:01<00:00, 2935.74iteration/s]


The truck n: 0
Best time: 41.87914952975381 h
Best Route: Cernay-l'Église -> Levis -> Jaignes -> Saint-Mard -> Boncourt -> Servais -> Warluis -> Courgent -> Saint-Georges-du-Rosay -> Saint-Denis-d'Augerons -> Bréauté -> Heudreville-sur-Eure -> Vatteville -> Linzeux -> Crèvec½ur-sur-l'Escaut -> Cernay-l'Église


Tabu Search: 100%|█████████████████████████████████████████████████████████████████████████| 3000/3000 [00:00<00:00, 18441.93iteration/s]


The truck n: 1
Best time: 24.201421266259278 h
Best Route: Cernay-l'Église -> Curtil-sous-Buffières -> Lacenas -> Sembadel -> Saint-Saturnin-de-Lenne -> Maruéjols-lès-Gardon -> Saint-Martin-le-Vinoux -> Champagny-en-Vanoise -> Cernay-l'Église


Tabu Search: 100%|█████████████████████████████████████████████████████████████████████████| 3000/3000 [00:00<00:00, 13407.53iteration/s]


The truck n: 2
Best time: 42.31188563132243 h
Best Route: Cernay-l'Église -> Uzech -> Saint-Michel-de-Vax -> Cierp-Gaud -> Issigeac -> Saint-Géraud -> Saint-Pierre-de-Bat -> Allas-Bocage -> Chapelle-Viviers -> Cernay-l'Église


Tabu Search: 100%|██████████████████████████████████████████████████████████████████████████| 3000/3000 [00:00<00:00, 3108.40iteration/s]


The truck n: 3
Best time: 23.070536027437164 h
Best Route: Cernay-l'Église -> Conte -> Le Frasnois -> Poinson-lès-Fayl -> Marnay-sur-Marne -> Bar-sur-Aube -> Villiers-en-Lieu -> Dannevoux -> Gravelotte -> Oron -> Oëlleville -> La Bourgonce -> Weinbourg -> Durrenbach -> Cleebourg -> Cernay-l'Église


Tabu Search: 100%|█████████████████████████████████████████████████████████████████████████| 3000/3000 [00:00<00:00, 26105.63iteration/s]

The truck n: 4
Best time: 51.23343658911645 h
Best Route: Cernay-l'Église -> Tessel -> Billé -> Marcillé-Robert -> Saint-Guinoux -> Plougastel-Daoulas -> Locqueltas -> Cernay-l'Église


# multi start implementation

In [15]:
def multi_start_optimization(time_matrix, truck_nb, map_obj, num_starts, use_traffic_matrix=False):
    best_routes = []
    best_times = []
    total_times = []
    trucks_list = []

    for i in tqdm(range(num_starts), desc="multistart"):
        time_matrix_shuffled = list(time_matrix.items())
        np.random.shuffle(time_matrix_shuffled)
        time_matrix_shuffled = dict(time_matrix_shuffled)
        
        trucks = split_into_trucks(time_matrix_shuffled, truck_nb)
        trucks_list.append(trucks)
        
        start_routes = []
        start_times = []

        for i, truck in enumerate(trucks):
            if use_traffic_matrix:
                truck = generate_traffic_matrix(truck)
            route, time = tabu_search(truck, num_iterations, tabu_list_size, start_town=next(iter(truck)), progress_enable=False)

            start_routes.append(route)
            start_times.append(time)

        best_routes.append(start_routes)
        best_times.append(start_times)
        total_time = sum(start_times)
        total_times.append(total_time)

    sorted_starts = sorted(range(num_starts), key=lambda k: total_times[k], reverse=True)
        
    for i, truck in enumerate(trucks_list[sorted_starts[-1]]):
        route = [lst for lst in best_routes[sorted_starts[-1]] if list(truck.keys())[1] in lst][0]
        plot_cities(filter_cities_by_truck(citiesSample, truck), route, map_obj=map_obj, color=color[i])
    
    print(f"the best route is: {best_routes[sorted_starts[-1]]}\n")
    print([f"truck{i + 1}: {time} h" for i, time in enumerate(best_times[sorted_starts[-1]])])
    print(f"\n with the slowest truck taking: {max(best_times[sorted_starts[-1]])} h")
    return map_obj

In [16]:
# with time constrain and trucks
map_obj = folium.Map(
    location=[
        float(citiesSample[0][2]), 
        float(citiesSample[0][3])
    ], 
    zoom_start=6)

multi_start_optimization(traffic_matrix, truck_nb, map_obj, num_starts)

multistart: 100%|████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:24<00:00,  2.46s/it]

the best route is: [['Saint-Michel-de-Vax', 'Saint-Saturnin-de-Lenne', 'Sembadel', 'Levis', 'Curtil-sous-Buffières', 'Lacenas', 'Saint-Martin-le-Vinoux', 'Maruéjols-lès-Gardon', 'Saint-Michel-de-Vax'], ['Saint-Michel-de-Vax', 'Saint-Georges-du-Rosay', "Saint-Denis-d'Augerons", 'Bréauté', 'Heudreville-sur-Eure', 'Vatteville', 'Courgent', 'Warluis', 'Linzeux', "Crèvec½ur-sur-l'Escaut", 'Servais', 'Boncourt', 'Saint-Mard', 'Jaignes', 'Saint-Michel-de-Vax'], ['Saint-Michel-de-Vax', 'Champagny-en-Vanoise', "Cernay-l'Église", 'La Bourgonce', 'Cleebourg', 'Durrenbach', 'Weinbourg', 'Oron', 'Oëlleville', 'Gravelotte', 'Dannevoux', 'Villiers-en-Lieu', 'Bar-sur-Aube', 'Marnay-sur-Marne', 'Poinson-lès-Fayl', 'Conte', 'Le Frasnois', 'Saint-Michel-de-Vax'], ['Saint-Michel-de-Vax', 'Tessel', 'Billé', 'Saint-Guinoux', 'Plougastel-Daoulas', 'Locqueltas', 'Marcillé-Robert', 'Saint-Michel-de-Vax'], ['Saint-Michel-de-Vax', 'Uzech', 'Cierp-Gaud', 'Saint-Géraud', 'Saint-Pierre-de-Bat', 'Allas-Bocage', 'Cha

# multi start implementation (with parallelism)

In [17]:
def multi_start_optimization(time_matrix, truck_nb, map_obj, num_starts, use_traffic_matrix=False, max_threads=4):
    best_routes = []
    best_times = []
    total_times = []
    trucks_list = []

    def optimize_start(start_index):
        time_matrix_shuffled = list(time_matrix.items())
        np.random.shuffle(time_matrix_shuffled)
        time_matrix_shuffled = dict(time_matrix_shuffled)

        trucks = split_into_trucks(time_matrix_shuffled, truck_nb)
        trucks_list.append(trucks)

        start_routes = []
        start_times = []

        for i, truck in enumerate(trucks):
            if use_traffic_matrix:
                truck = generate_traffic_matrix(truck)

            route, time = tabu_search(truck, num_iterations=100, tabu_list_size=100, start_town=next(iter(truck)), progress_enable=False)

            start_routes.append(route)
            start_times.append(time)
            
            progress_bar.update(1)  # Increment the common progress bar

        best_routes.append(start_routes)
        best_times.append(start_times)
        total_time = sum(start_times)
        total_times.append(total_time)

    # Create a single common progress bar
    progress_bar = tqdm(total=num_starts * truck_nb, desc="parallelised multistart", position=0)

    with concurrent.futures.ThreadPoolExecutor(max_workers=max_threads) as executor:
        futures = [executor.submit(optimize_start, i) for i in range(num_starts)]
        concurrent.futures.wait(futures)

    progress_bar.close()  # Close the common progress bar

    sorted_starts = sorted(range(num_starts), key=lambda k: total_times[k], reverse=True)

    for i, truck in enumerate(trucks_list[sorted_starts[-1]]):
        route = [lst for lst in best_routes[sorted_starts[-1]] if list(truck.keys())[1] in lst][0]
        plot_cities(filter_cities_by_truck(citiesSample, truck), route, map_obj=map_obj, color=color[i])

    print(f"the best route is: {best_routes[sorted_starts[-1]]}\n")
    print([f"truck{i + 1}: {time} h" for i, time in enumerate(best_times[sorted_starts[-1]])])
    print(f"\n with the slowest truck taking: {max(best_times[sorted_starts[-1]])} h")
    return map_obj

In [ ]:
# with time constrain and trucks
map_obj = folium.Map(
    location=[
        float(citiesSample[0][2]), 
        float(citiesSample[0][3])
    ], 
    zoom_start=6)

multi_start_optimization(traffic_matrix, truck_nb, map_obj, num_starts)